In [9]:
import barcode
from barcode.writer import ImageWriter, SVGWriter
import treepoem

from io import BytesIO
from barcode import Code39


In [4]:
# Escolha o tipo de código de barras que deseja gerar
barcode_type = 'ean13'  # EAN-13 é um dos tipos de códigos de barras suportados

# Número do código de barras (deve ser um número válido para o tipo escolhido)
barcode_number = '100000000001'  # Deve ter 12 dígitos para EAN-13

# Gerar o código de barras
barcode_class = barcode.get_barcode_class(barcode_type)
barcode_instance = barcode_class(barcode_number, writer=ImageWriter())

# Salvar o código de barras em um arquivo
filename = barcode_instance.save('barcode/barcode')

print(f'Código de barras gerado e salvo como {filename}.png')

Código de barras gerado e salvo como barcode/barcode.png.png


In [12]:
# Write to a file-like object:
rv = BytesIO()
Code39("08191442906", writer=SVGWriter()).write(rv)

# Or to an actual file:
with open("barcode/somefile.svg", "wb") as f:
    Code39(str('08191442906'), writer=SVGWriter()).write(f)